In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/us-accidents/US_Accidents_June20.csv")
df.head()


In [ ]:
df.describe()
df.isnull().sum()

In [ ]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = df[df.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

In [ ]:
# let's examine the types of the column labels
all(isinstance(column, str) for column in df.columns)

In [ ]:
accidents_by_city = df.groupby('City')['ID'].count().reset_index(name="count")
#accidents_by_city.head()
top10=accidents_by_city.nlargest(10,'count')
top10.head()

In [ ]:
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot')
import seaborn as sns

In [ ]:
plt.xticks(rotation='vertical')
plt.bar(x = accidents_by_city[accidents_by_city['count']>20000]['City'],height=accidents_by_city[accidents_by_city['count']>20000]['count'])

In [ ]:
df.dropna(subset=['Severity'])
df.dropna(subset=['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)','Pressure(in)','Visibility(mi)', 'Wind_Direction','Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition'], how='all')
accidents_by_city_s = df[df['Severity']==4].groupby('City')['ID'].count().sort_values(ascending=False).reset_index(name="count")
accidents_by_city_s.nlargest(10,'count').head()
#grouped = top10.groupby(['City', 'Severity'], as_index='false')['City'].count().sort_values(ascending=False)
#grouped.unstack().plot(kind='bar', stacked=True)
#grouped.head()

In [ ]:
plt.xticks(rotation='vertical')
plt.bar(x = accidents_by_city_s[accidents_by_city_s['count']>500]['City'],height=accidents_by_city_s[accidents_by_city_s['count']>500]['count'])

In [ ]:
#df[df['City']=="Miami"].nlargest(10,'End_Lat').head(10)
df[df['City']=="Miami"].describe()

In [ ]:
df.dropna(subset=['Temperature(F)','Wind_Chill(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction','Wind_Speed(mph)','Precipitation(in)','Weather_Condition',], how='all')
accidents_by_severity = df.groupby('Severity')['ID'].count().reset_index(name="count1")
accidents_by_severity['Severity']= accidents_by_severity['Severity'].astype(str)
accidents_by_severity.head()

In [ ]:
df_vis=df[['Severity','Visibility(mi)']]
#df_vis.plot(kind='area', stacked=False, figsize=(20, 10), # pass a tuple (x, y) size)

#plt.title('Immigration Trend of Top 5 Countries')
#plt.ylabel('Number of Immigrants')
#plt.xlabel('Years')

#plt.show()
df_vis.head()

In [ ]:
plt.scatter( 'Visibility(mi)','Severity', data=df_vis)
plt.xlabel('Visibity')
plt.ylabel('Severity')
plt.show()

In [ ]:
df_R1=df[df['Severity'] !=1]
#df_R1=df.drop([df['Severity']==1].index, inplace=True)
df_R1.describe()

In [ ]:
df_2=df[df['Severity']==2].sample(n=112320)
df_3=df[df['Severity']==3].sample(n=112320)
df_4=df[df['Severity']==4]
df_norm=pd.concat([df_2,df_3,df_4])
df_norm.describe()

In [ ]:
plt.scatter( 'Visibility(mi)','Severity', data=df_norm)
plt.xlabel('Visibity')
plt.ylabel('Severity')
plt.show()

In [ ]:
plt.scatter( 'Precipitation(in)','Severity',data=df_norm)
plt.xlabel('Precipitation')
plt.ylabel('Severity')
plt.show()

In [ ]:
all(isinstance(column, str) for column in df_norm.columns)

In [ ]:
df_norm.head(10)
df_norm.dropna(subset=['TMC'])
plt.scatter( 'Severity','TMC',data=df_norm)
plt.ylabel('TMC')
plt.xlabel('Severity')
plt.show()

In [ ]:
df_norm.Weather_Condition.unique()
accidents_by_Wcondition = df_norm.groupby(['Weather_Condition','Severity'], as_index='false')['ID'].count().sort_values(ascending=False).reset_index(name="count")
accidents_by_Wcondition.nlargest(20,'count').head(20)

In [ ]:
df_d=df_norm[df_norm['Traffic_Signal']==True][['Distance(mi)','Severity']]
df_d.shape

In [ ]:
df_norm.corr(method='pearson')

In [ ]:
import scipy.stats
sns.regplot(x="Distance(mi)",y="Severity",data=df_norm)
plt.ylim(0,)

In [ ]:

p_value=scipy.stats.pearsonr(df_norm['Distance(mi)'],df_norm['Severity'])
print(p_value)

In [ ]:
df_d=df_norm[df_norm['Traffic_Signal']==True][['Distance(mi)','Severity']]
sns.regplot(x="Distance(mi)",y="Severity",data=df_d)
plt.ylim(0,)

In [ ]:
p_value=scipy.stats.pearsonr(df_d['Distance(mi)'],df_d['Severity'])
print(p_value)

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
X,y=make_classification(random_state=0)
pipe=Pipeline([('scaler',StandardScaler()),('polynomial',PolynomialFeatures(degree=2)),'mode',LinearRegression()])
z=df_norm[['TMC','Distance(mi)','Junction','Traffic_Signal','Crossing']]
pipe.fit(df_norm[['TMC','Distance(mi)','Junction','Traffic_Signal','Crossing']],y)
Yhat=pipe.predict(X[['TMC','Distance(mi)','Junction','Traffic_Signal','Crossing']])
#x_scale=scale.transform(df_norm[['TMC','Distance(mi)','Junction','Traffic_Signal','Crossing']])
#pr= PolynomialFeatures(degree=2)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df_norm['Severity'],Y_predict_simple_fit)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
x_train,x_test,y_train,y_test=train_test_split(z,df_norm[['Severity'],test_size=0.3,random_state=0)
scores=cross_val_score(lr,z,df_norm[['Severity'],cv=3)
np.mean(scores)
yhat=cross_val.predict(lr2e,z,df_norm[['Severity'],cv=3)

In [ ]:
from sklearn.linear_model import Ridge
Ridgemodel = Ridge(alpha = 0.1)
RidgeModel.fit(X,y)
yhat=RidgeModel.predict(X)
parameters=[{'alpha'}:[1,10,100,1000] }]

In [ ]:
from sklearn.linear_model import GridSearchCV
parameters=[{'alpha'}:[1,10,100,1000,10000,100000] }]
RR=Ridge()
Grid1=GridSearchCV(RR,parameters1,cv=4)
Grid1.fit(z,df_norm[['Severity'])
Grid1_best_estimator_
scores=Grid1.cv.results